In [4]:
pip install undetected-chromedriver

In [5]:
pip install futures

In [1]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from functools import reduce

import mysql.connector as mysql
from mysql.connector import Error

import pandas as pd
import time
import datetime
import re

from concurrent.futures import ThreadPoolExecutor

In [1]:
data = pd.read_csv("D:/Images2/Date_FS.csv")
print(data)

conn = mysql.connect(host='localhost', database='****', user='newuser', password='l******d', auth_plugin='mysql_native_password')
if conn.is_connected():
    cursor = conn.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()

'data = pd.read_csv("D:/Images2/Date_FS.csv")\nprint(data)\n\nconn = mysql.connect(host=\'localhost\', database=\'****\', user=\'newuser\', password=\'l******d\', auth_plugin=\'mysql_native_password\')\nif conn.is_connected():\n    cursor = conn.cursor()\n    cursor.execute("select database();")\n    record = cursor.fetchone()'

In [2]:
options = uc.ChromeOptions()
options.add_argument("start-maxmized")
driver = uc.Chrome(options=options)

rows = []

for i in data['URLs']:
    row = {}
    
    driver.get(i)
    time.sleep(3)
    
    date = driver.find_element(By.XPATH, "//div[@class='data1Css_dhsszr2 typeBaseCss_tf6xlml']").text
    
    row['URL'] = i
    row['descriptions'] = date
    
    #rows.append(row)
    
    sql = 'INSERT INTO `new_table` VALUES(%s, %s)'
    cursor.execute(sql, tuple(row.values()))
    
    conn.commit()
    
    print(tuple(row.values()))


'options = uc.ChromeOptions()\noptions.add_argument("start-maxmized")\ndriver = uc.Chrome(options=options)\n\nrows = []\n\nfor i in data[\'URLs\']:\n    row = {}\n    \n    driver.get(i)\n    time.sleep(3)\n    \n    date = driver.find_element(By.XPATH, "//div[@class=\'data1Css_dhsszr2 typeBaseCss_tf6xlml\']").text\n    \n    row[\'URL\'] = i\n    row[\'descriptions\'] = date\n    \n    #rows.append(row)\n    \n    sql = \'INSERT INTO `new_table` VALUES(%s, %s)\'\n    cursor.execute(sql, tuple(row.values()))\n    \n    conn.commit()\n    \n    print(tuple(row.values()))'

In [3]:
def extract_info(url):
    
    print("extracting: ", url)
    
    options = uc.ChromeOptions()
    options.add_argument("start-maxmized")
    #options.add_argument("--headless")
    driver = uc.Chrome(options=options)

    rows = []
    for i in url:
    
        row = {}

        driver.get(i)
        time.sleep(3)

        date = driver.find_element(By.XPATH, "//div[@class='data1Css_dhsszr2 typeBaseCss_tf6xlml']").text

        row['URL'] = i
        row['descriptions'] = date

        #rows.append(row)

        sql = 'INSERT INTO `new_table` VALUES(%s, %s)'
        cursor.execute(sql, tuple(row.values()))

        conn.commit()

        print("inserted into sql")


urls = data['URLs'].to_list()

with ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(extract_info, data['URLs'])


'def extract_info(url):\n    \n    print("extracting: ", url)\n    \n    options = uc.ChromeOptions()\n    options.add_argument("start-maxmized")\n    #options.add_argument("--headless")\n    driver = uc.Chrome(options=options)\n\n    rows = []\n    for i in url:\n    \n        row = {}\n\n        driver.get(i)\n        time.sleep(3)\n\n        date = driver.find_element(By.XPATH, "//div[@class=\'data1Css_dhsszr2 typeBaseCss_tf6xlml\']").text\n\n        row[\'URL\'] = i\n        row[\'descriptions\'] = date\n\n        #rows.append(row)\n\n        sql = \'INSERT INTO `new_table` VALUES(%s, %s)\'\n        cursor.execute(sql, tuple(row.values()))\n\n        conn.commit()\n\n        print("inserted into sql")\n\n\nurls = data[\'URLs\'].to_list()\n\nwith ThreadPoolExecutor(max_workers=2) as executor:\n    executor.map(extract_info, data[\'URLs\'])'